In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'

In [3]:
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
files

,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
0,name/ Product_stats.csv,name,string,0.994361,Product_stats.csv,432728.0,product_name
1,offers/ Product_stats.csv,offers,string,0.964249,Product_stats.csv,419624.0,offer
2,brand/ Product_stats.csv,brand,string,0.403728,Product_stats.csv,175695.0,product_name
3,headline/ CreativeWork_stats.csv,headline,string,0.939871,CreativeWork_stats.csv,76451.0,headline
4,author/ CreativeWork_stats.csv,author,string,0.776499,CreativeWork_stats.csv,63162.0,person_name
...,...,...,...,...,...,...,...
202,numtracks/ MusicAlbum_stats.csv,numtracks,int,0.415808,MusicAlbum_stats.csv,121.0,number
203,menu/ Restaurant_stats.csv,menu,string,0.104581,Restaurant_stats.csv,121.0,text_description
204,contactpoint/ Person_stats.csv,contactpoint,string,0.009353,Person_stats.csv,120.0,text_description
205,email/ Hotel_stats.csv,email,string,0.107664,Hotel_stats.csv,118.0,email


In [4]:
columns_sorted = files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True)
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [26]:
inputdir = '../../src/data/schemafiltereddata/Book/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
books = pd.DataFrame ()
setcol = set(columns_sorted.loc['Book']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    list_numNAs = list(df[df.columns[df.columns.isin(setcol)]].isnull().sum())
    list_col = list(df[df.columns[df.columns.isin(setcol)]])
    books = books.append({'Class':'Book','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': list_numNAs, 'columns':list_col}, ignore_index=True)
books = books.explode(['columns', 'numNAs'])
#books = books.explode('numNAs')
#books = books[books['columns'].isin(setcol)]

display(books)

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,"[name, author, publisher, offers, bookedition]",Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0
1,Book,"[name, author, publisher, isbn, datepublished,...",Book_virago.co.uk_September2020.json.gz,"[0, 2, 0, 0, 0, 1]",11.0,6.0,398.0
2,Book,"[name, author, publisher, offers, bookedition]",Book_borgantiquarian.com_September2020.json.gz,"[0, 0, 157, 0, 159]",8.0,5.0,248.0
3,Book,"[name, author, publisher, isbn, datepublished]",Book_downpour.com_September2020.json.gz,"[0, 13, 0, 7323, 22]",8.0,5.0,12194.0
4,Book,"[name, author, publisher, isbn, datepublished,...",Book_marasworld.com_September2020.json.gz,"[0, 0, 0, 0, 0, 0]",8.0,6.0,26.0
...,...,...,...,...,...,...,...
973,Book,"[name, publisher, isbn, bookformat, datepublis...",Book_benjamins.com_September2020.json.gz,"[0, 7, 756, 755, 62, 775, 8, 0]",14.0,8.0,1795.0
974,Book,"[name, author, publisher, isbn, datepublished,...",Book_nollegiu.com_September2020.json.gz,"[0, 77, 0, 36, 1465, 0, 561]",10.0,7.0,3175.0
975,Book,"[name, author, publisher, isbn, datepublished,...",Book_nosegraze.com_September2020.json.gz,"[0, 0, 0, 109, 0, 1, 1]",9.0,7.0,153.0
976,Book,"[name, author, publisher, isbn, datepublished,...",Book_hachettebooks.com_September2020.json.gz,"[0, 0, 0, 0, 0, 45, 1]",12.0,7.0,583.0


In [50]:
books_columns = books['columns'].explode('columns')
books_nas = books['numNAs'].explode('numNAs')

print(books_columns)
print(books_nas)

0                name
1              author
2           publisher
3              offers
4         bookedition
            ...      
6452             isbn
6453       bookformat
6454    datepublished
6455           offers
6456    numberofpages
Name: columns, Length: 6457, dtype: object
0         1
1        40
2       157
3         0
4       138
       ... 
6452    385
6453      0
6454    135
6455      0
6456    316
Name: numNAs, Length: 6457, dtype: object


In [53]:
together = pd.concat([books_columns, books_nas], axis=1)
together

,columns,numNAs
0,name,1
1,author,40
2,publisher,157
3,offers,0
4,bookedition,138
...,...,...
6452,isbn,385
6453,bookformat,0
6454,datepublished,135
6455,offers,0


In [62]:
for col in books.columns:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    na_columns=df[df.columns].isnull().sum()
    books = books.append({'number_of_NAs':na_columns}, ignore_index = True)
books

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows,number_of_NAs
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,NaN
1,Book,author,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,NaN
2,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,NaN
3,Book,offers,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,NaN
4,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,"[1, 40, 157, 0, 138]",8.0,5.0,274.0,NaN
...,...,...,...,...,...,...,...,...
6459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,row_id 0 name 0 desc...
6460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,row_id 0 name 0 desc...
6461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,row_id 0 name 0 desc...
6462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,row_id 0 name 0 desc...


In [23]:
inputdir = '../../src/data/schemafiltereddata/CreativeWork/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
cw = pd.DataFrame ()
setcol = set(columns_sorted.loc['CreativeWork']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    list_numNAs = df[df.columns[df.columns.isin(setcol)]].isnull().sum().sum()
    list_col = list(df.columns)
    cw = cw.append({'Class':'CreativeWork','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': list_numNAs, 'columns':list_col}, ignore_index=True)
cw = cw.explode('columns')
#books = books.explode('numNAs')
cw = cw[cw['columns'].isin(setcol)]

display(cw)

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,CreativeWork,headline,CreativeWork_greatrooms24.com_September2020.js...,3.0,10.0,7.0,171.0
0,CreativeWork,author,CreativeWork_greatrooms24.com_September2020.js...,3.0,10.0,7.0,171.0
0,CreativeWork,datepublished,CreativeWork_greatrooms24.com_September2020.js...,3.0,10.0,7.0,171.0
0,CreativeWork,datemodified,CreativeWork_greatrooms24.com_September2020.js...,3.0,10.0,7.0,171.0
0,CreativeWork,name,CreativeWork_greatrooms24.com_September2020.js...,3.0,10.0,7.0,171.0
...,...,...,...,...,...,...,...
5517,CreativeWork,author,CreativeWork_manolisheliotis.co.uk_September20...,0.0,8.0,5.0,14.0
5517,CreativeWork,datepublished,CreativeWork_manolisheliotis.co.uk_September20...,0.0,8.0,5.0,14.0
5517,CreativeWork,datemodified,CreativeWork_manolisheliotis.co.uk_September20...,0.0,8.0,5.0,14.0
5517,CreativeWork,publisher,CreativeWork_manolisheliotis.co.uk_September20...,0.0,8.0,5.0,14.0


In [16]:
print(cw[cw['num_rows']<=10])

             Class        columns  \
1840  CreativeWork       headline   
1840  CreativeWork  datepublished   
1840  CreativeWork   datemodified   
1840  CreativeWork   commentcount   
1864  CreativeWork       headline   
...            ...            ...   
5495  CreativeWork   datemodified   
5495  CreativeWork   commentcount   
5509  CreativeWork           name   
5509  CreativeWork       keywords   
5509  CreativeWork          genre   

                                               filename        numNAs  \
1840  CreativeWork_rayscatering.com_September2020.js...  [0, 0, 0, 0]   
1840  CreativeWork_rayscatering.com_September2020.js...  [0, 0, 0, 0]   
1840  CreativeWork_rayscatering.com_September2020.js...  [0, 0, 0, 0]   
1840  CreativeWork_rayscatering.com_September2020.js...  [0, 0, 0, 0]   
1864  CreativeWork_yuspecialtycafe.com_September2020...     [0, 0, 0]   
...                                                 ...           ...   
5495      CreativeWork_adgens.com_September2

In [15]:
inputdir = '../../src/data/schemafiltereddata/Book/'
files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
books = pd.DataFrame ()
setcol = set(columns_sorted.loc['Book']['listofcolumns'])
for file in files:
    df = pd.read_json(os.path.join(inputdir+file), compression = 'gzip', lines=True)
    numrows = df.shape[0]
    numcolumns = df.shape[1]
    numselcol = df[df.columns[df.columns.isin(setcol)]].shape[1]
    numNAs = df[df.columns[df.columns.isin(setcol)]].isnull().sum().sum()
    list_col = list(df.columns)
    books = books.append({'Class':'Book','filename':file,'num_rows':numrows, 'num_columns': numcolumns, 'num_columns_selected':numselcol, 'numNAs': numNAs, 'columns':list_col}, ignore_index=True)
books = books.explode('columns')
books = books[books['columns'].isin(setcol)]
display(books)

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows
0,Book,name,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,author,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,publisher,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,offers,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
0,Book,bookedition,Book_foldvaribooks.com_September2020.json.gz,336.0,8.0,5.0,274.0
...,...,...,...,...,...,...,...
977,Book,isbn,Book_cogito-shop.com_September2020.json.gz,1618.0,11.0,8.0,3498.0
977,Book,bookformat,Book_cogito-shop.com_September2020.json.gz,1618.0,11.0,8.0,3498.0
977,Book,datepublished,Book_cogito-shop.com_September2020.json.gz,1618.0,11.0,8.0,3498.0
977,Book,offers,Book_cogito-shop.com_September2020.json.gz,1618.0,11.0,8.0,3498.0


In [22]:
allRows_selCol = books.num_columns*books.num_rows
allRows_selCol
pOfAll = allRows_selCol/numNAs
pOfAll
#books = books.append({'percentage_of_all':})

0       1.354759
0       1.354759
0       1.354759
0       1.354759
0       1.354759
         ...    
977    23.781211
977    23.781211
977    23.781211
977    23.781211
977    23.781211
Length: 6457, dtype: float64

In [8]:
books = books[books['numNAs'].isin(setcol)]
books

,Class,columns,filename,numNAs,num_columns,num_columns_selected,num_rows


In [ ]:
#function to count rows and move tables that have less than 10 rows
def count_move_small_tables(table):
    df = pd.read_json(os.path.join(inputdir+table), compression='gzip', lines=True)
    if len(df)<=10:
        df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        #os.remove(os.path.join(output, '{}'.format(file)))
        print(len(df))
    elif df[df[col for col in setcol].isna()]:
        
        #df.to_json(os.path.join(outputdir+table), orient = 'records', compression = 'gzip', lines = True)
        #os.remove(os.path.join(output, '{}'.format(file)))
        return
    else:
        row_df = pd.DataFrame ()
        row_df = row_df.append({'file': table, 'length': len(df)}, ignore_index=True)
    print(table, len(files))
    print(row_df)
    #print(row_df)

In [ ]:
#files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
#run function for all classes & tables with multiprocessing
#row_df = pd.DataFrame ()
if __name__ == '__main__':
    for index in columns_sorted.filename:
        print(index)
        setcol = set(columns_sorted.loc[index]['listofcolumns'])
        print(setcol)
        inputdir = '../../src/data/schemafiltereddata/'+index+'/'
        outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows_and_NAs/'
        if not os.path.exists(outputdir):
            os.makedirs(outputdir)
        p = Pool(100)
        for _ in tqdm(p.imap_unordered(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]), total=len([file for file in os.listdir(inputdir) if file.endswith('.json.gz')])):
            pass
        p.map(count_move_small_tables, [file for file in os.listdir(inputdir) if file.endswith('.json.gz')])
        p.close()
        #print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
        #new_df = pd.DataFrame ()
        #new_df = new_df.append({'file': row_df['filename'.groupby(index))], 'mean_length': row_df.mean(len(df))}, ignore_index=True)
        p.join()

In [ ]:
print(row_df)

In [ ]:
row_df.mean(axis = 1).mean()

In [ ]:
num_tables=[]
for index in columns_sorted.filename:
    print(index)
    outputdir = '../../src/data/schemafiltereddata/'+index+'/under_10_rows_and_NAs/'
    print(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
    num_tables.append(len([file for file in os.listdir(outputdir) if file.endswith('.json.gz')]))
len(num_tables)

In [ ]:
test_df = pd.read_json('/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/schemafiltereddata/CreativeWork/under_10_rows/CreativeWork_1daywebs.com_September2020.json.gz', compression='gzip', lines=True)
test_df

In [ ]:
overview = pd.read_csv(path+'/filestats_over_all_columns.csv')
overview

In [ ]:
print(overview[(overview['num_rows']<=10) & (overview['Class']=='CreativeWork')])

In [ ]:
overview_tables = pd.DataFrame()
for index in columns_sorted.filename:
    round = pd.DataFrame()
    print(index)
    print(round)
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]
    for file in files:
        # read file
        df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        numrows= df.shape[0]
        numcolumns = df.shape[1]
        list_col = list(df.columns)
        
        round = round.append({'Class':index,'filename':file,'num_rows':numrows, 'num_columns':numcolumns, 'columns':list_col}, ignore_index=True)
    liste = set(listcolt.loc[index]['listofcolumns'])
    round = round.explode('columns')
    round = round[round['columns'].isin(liste)]
    overview = overview.append(round)
    display(overview)